In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import os
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader

from torch.optim import AdamW
import torch.nn.functional as F
import torch
import json 
import os
import wandb
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 6.9 MB/s eta 0:00:00:00:01


# Features table of product items

### This table contains all h&m articles with details such as a type of product, a color, a product group and other features.  
**Article data description:**

- `article_id` : A unique identifier of every article.
- `product_code`, `prod_name` : A unique identifier of every product and its name (not the same).
- `product_type`, `product_type_name` : The group of product_code and its name.
- `product_group_name` : Product Group. Father to product type.
- `graphical_appearance_no`, `graphical_appearance_name` : The group of graphics and its name.
- `colour_group_code`, `colour_group_name` : The group of color and its name.
- `perceived_colour_value_id`, `perceived_colour_value_name`, `perceived_colour_master_id`, `perceived_colour_master_name` : The added color info.
- `department_no`, `department_name` : A unique identifier of every department and its name.
- `index_code`, `index_name` : A unique identifier of every index and its name.
- `index_group_no`, `index_group_name` : A group of indices and its name.
- `section_no`, `section_name` : A unique identifier of every section and its name.
- `garment_group_no`, `garment_group_name` : A unique identifier of every garment and its name.
- `detail_desc` : Details.

In [2]:
text_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv'
articles = pd.read_csv(text_path)
print(articles.shape) # 100k data points
articles.head(5)

(105542, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


# LoRA finetune CLIP (single column) (ignored)

## Dataset on product_group_name 

In [3]:
# import os
# import pandas as pd
# from datasets import Dataset
# import itertools

# # 图片文本目录路径
# images_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/images'
# text_path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv'

# # 读取csv文件
# # df = pd.read_csv(text_path)
# df = articles
# df['image_path'] = ''

# # 为每个article_id添加对应的图片路径，并将表格保存到/kaggle/working/articles_with_image_path.csv
# for i in range(df.shape[0]):
#     # 用article_id获取图片路径
#     article_id = str(df.iloc[i]['article_id'])
#     image_path = images_path + f'/0{article_id[:2]}' + f'/0{article_id}.jpg'
#     if not os.path.exists(image_path):
#         continue
#     df.loc[i, 'image_path'] = image_path
    

# df.to_csv('/kaggle/working/articles_with_image_path.csv')

In [4]:
# # remove all the columns that their image_path is NAN
# df = pd.read_csv('/kaggle/working/articles_with_image_path.csv')
# filter_product_group_name = ['Unknown','Underwear/nightwear','Cosmetic','Bags','Items',
#     'Furniture','Garment and Shoe care','Stationery','Interior textile','Fun']
# format_df = df[df['image_path'].notna() & (df['image_path'] != '') & (~df['product_group_name'].isin(filter_product_group_name))]
# print(f'清洗后的df行列数: {format_df.shape}')

# # 将df拆分为测试集df和训练集df，并保证product_group_name类别比例保持一致。
# from sklearn.model_selection import StratifiedShuffleSplit
# import pandas as pd
# import numpy as np

# def stratified_split(df, text_column, test_size=0.2, random_state=42):
#     # 为每个唯一的 text 值分配一个类别标签
#     df['text_category'] = pd.Categorical(df[text_column]).codes
    
#     # 初始化 StratifiedShuffleSplit
#     splitter = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    
#     # 进行分层抽样
#     for train_index, test_index in splitter.split(df, df['text_category']):
#         train_df = df.iloc[train_index].copy()
#         test_df = df.iloc[test_index].copy()
    
#     # 删除临时的 'text_category' 列
#     train_df.drop('text_category', axis=1, inplace=True)
#     test_df.drop('text_category', axis=1, inplace=True)
    
#     return train_df, test_df

# # 进行分层抽样
# train_df, val_df = stratified_split(format_df, text_column='product_group_name', test_size=0.2)

# print(f"训练集大小: {len(train_df)}")
# print(f"验证集大小: {len(val_df)}")

# # 检查每个集合中各类别的比例
# def check_proportions(df, column):
#     return df[column].value_counts(normalize=True)

# print("\n训练集中的类别比例:")
# print(check_proportions(train_df, 'product_group_name'))

# print("\n验证集中的类别比例:")
# print(check_proportions(val_df, 'product_group_name'))
# # Unknown之后的占比太小（只占了300个dp不到）感觉可以删除，但如何处理这部分的推理

# Multi labels training

## Model loading and lora config

In [2]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import (
    CLIPProcessor, 
    CLIPModel, 
    TrainingArguments, 
    Trainer
)

# Load CLIP model and processor

model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", 'k_proj'],
    lora_dropout=0.05,
    bias="none",
#     task_type="classification"
)

peft_model = get_peft_model(model, lora_config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device)
print(peft_model.print_trainable_parameters())

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


trainable params: 1,474,560 || all params: 152,751,873 || trainable%: 0.9653
None


## Data processing and Dataset 

In [3]:
df = pd.read_csv('/kaggle/input/articles-with-image-path/articles_with_image_path (1).csv')
df = df[df['image_path'].notna() & (df['image_path'] != '')]
target_columns = ["product_group_name", "product_type_name", "graphical_appearance_name", "colour_group_name", "perceived_colour_value_name", "perceived_colour_master_name", "department_name", "index_name", "index_group_name", "section_name", "garment_group_name"]
# 11 target col
print(df.shape)
df.head()

(105100, 27)


,Unnamed: 0,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,image_path
0,0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,...,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,/kaggle/input/h-and-m-personalized-fashion-rec...
1,1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,...,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,/kaggle/input/h-and-m-personalized-fashion-rec...
2,2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,...,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,/kaggle/input/h-and-m-personalized-fashion-rec...
3,3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,...,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",/kaggle/input/h-and-m-personalized-fashion-rec...
4,4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,...,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",/kaggle/input/h-and-m-personalized-fashion-rec...


In [4]:
def split_dataset(df, val_ratio=0.2, seed=42):
    """
    将DataFrame划分为训练集和验证集
    
    Args:
        df: 输入的DataFrame
        val_ratio: 验证集占比，默认0.2
        seed: 随机种子，用于复现结果
        
    Returns:
        train_df: 训练集DataFrame
        val_df: 验证集DataFrame
    """
    # 设置随机种子
    np.random.seed(seed)
    
    # 获取总行数
    n_total = len(df)
    
    # 计算验证集大小
    n_val = int(n_total * val_ratio)
    
    # 生成随机索引
    indices = np.random.permutation(n_total)
    val_indices = indices[:n_val]
    train_indices = indices[n_val:]
    
    # 划分数据集
    val_df = df.iloc[val_indices].copy().reset_index(drop=True)
    train_df = df.iloc[train_indices].copy().reset_index(drop=True)
    
    print(f"总数据量: {n_total}")
    print(f"训练集大小: {len(train_df)} ({1-val_ratio:.1%})")
    print(f"验证集大小: {len(val_df)} ({val_ratio:.1%})")
    
    return train_df, val_df

train_df, val_df = split_dataset(df, val_ratio=0.2)

columns_unique_labels = {}
for t_col in target_columns:
    columns_unique_labels[t_col] = list(df[t_col].unique())
for k,v in columns_unique_labels.items():
    print(f'\nClass {k} has {len(v)} unique labels:')
    print(v)

总数据量: 105100
训练集大小: 84080 (80.0%)
验证集大小: 21020 (20.0%)

Class product_group_name has 19 unique labels:
['Garment Upper body', 'Underwear', 'Socks & Tights', 'Garment Lower body', 'Accessories', 'Items', 'Nightwear', 'Unknown', 'Underwear/nightwear', 'Shoes', 'Swimwear', 'Garment Full body', 'Cosmetic', 'Interior textile', 'Bags', 'Furniture', 'Garment and Shoe care', 'Fun', 'Stationery']

Class product_type_name has 131 unique labels:
['Vest top', 'Bra', 'Underwear Tights', 'Socks', 'Leggings/Tights', 'Sweater', 'Top', 'Trousers', 'Hair clip', 'Umbrella', 'Pyjama jumpsuit/playsuit', 'Bodysuit', 'Hair string', 'Unknown', 'Hoodie', 'Sleep Bag', 'Hair/alice band', 'Belt', 'Boots', 'Bikini top', 'Swimwear bottom', 'Underwear bottom', 'Swimsuit', 'Skirt', 'T-shirt', 'Dress', 'Hat/beanie', 'Kids Underwear top', 'Shorts', 'Shirt', 'Cap/peaked', 'Pyjama set', 'Sneakers', 'Sunglasses', 'Cardigan', 'Gloves', 'Earring', 'Bag', 'Blazer', 'Other shoe', 'Jumpsuit/Playsuit', 'Sandals', 'Jacket', 'Cos

In [5]:
class TrainDataset(Dataset):
    def __init__(self, dataframe, target_columns_list):
        self.dataframe = dataframe
        self.t_columns_list = target_columns_list
        self.prompt = 'The {column} of photo is {label}' # temporary, need to be changed
            
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        image = Image.open(row['image_path'])
        
        text_list = []
        for t_cols in self.t_columns_list:
            labels = [row[col].strip() for col in t_cols]
            text = '. '.join([self.prompt.format(column=' '.join(c.split('_')), label=l) for c, l in zip(t_cols, labels)])
            text_list.append(text)
        # select the labels from k columns to create the prompt
        return {
            'image': image,
            'text_list': text_list,
        }
    
# Define data collator for training dataset
def collate_fn_train(batch):
#     print(batch)
#     一个dp包含了len(t_columns_list)个图片
    texts = []
    images = []
    for item in batch:
        texts.extend(item['text_list'])
        images.extend([item['image']] * len(item['text_list']))
    
    # 处理文本
    inputs = processor(
        text=texts,
        images=images,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

    # 合并文本和图像的输入
    inputs = {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'pixel_values': inputs['pixel_values'],
    }
    
    return inputs


from torch.utils.data import Dataset, ConcatDataset

t_columns_list = [
    ["product_group_name", "product_type_name", "graphical_appearance_name"],
    ["colour_group_name", "perceived_colour_value_name", "perceived_colour_master_name"],
    ["department_name", "index_name", "index_group_name"],
    ["section_name", "garment_group_name"],
] 
# 重复构造train_ds再连接起来
# train_ds_list = []
# for t_cols in t_columns_list:
#     train_ds = TrainDataset(train_df, t_cols)
#     train_ds_list.append(train_ds)

# train_ds = ConcatDataset(train_ds_list)

train_ds = TrainDataset(train_df, t_columns_list)
print(len(train_ds))
for i in range(0, 2):
    print(train_ds[i])

84080
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1166x1750 at 0x78BC77895DE0>, 'text_list': ['The product group name of photo is Garment Full body. The product type name of photo is Dress. The graphical appearance name of photo is Check', 'The colour group name of photo is Light Purple. The perceived colour value name of photo is Dusty Light. The perceived colour master name of photo is Lilac Purple', 'The department name of photo is Dresses. The index name of photo is Divided. The index group name of photo is Divided', 'The section name of photo is Divided Collection. The garment group name of photo is Dresses Ladies']}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1470x1750 at 0x78BC778958D0>, 'text_list': ['The product group name of photo is Garment Upper body. The product type name of photo is Hoodie. The graphical appearance name of photo is Solid', 'The colour group name of photo is Grey. The perceived colour value name of photo is Dusty L

In [6]:
class ValidateDataset(Dataset):
    def __init__(self, dataframe, target_columns):
        self.dataframe = dataframe
        self.target_columns = target_columns

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
#         image = Image.open(row['image_path'])
        dp = [row['image_path']]
        dp += [row[col] for col in self.target_columns]
        return dp
    
val_ds = ValidateDataset(val_df, target_columns)
for i in range(2):
    print(val_ds[i])

['/kaggle/input/h-and-m-personalized-fashion-recommendations/images/055/0559129003.jpg', 'Garment Lower body', 'Trousers', 'Denim', 'Blue', 'Medium Dusty', 'Blue', 'Young Boy Denim', 'Children Sizes 134-170', 'Baby/Children', 'Young Boy', 'Trousers Denim']
['/kaggle/input/h-and-m-personalized-fashion-recommendations/images/050/0500262002.jpg', 'Garment Upper body', 'Blazer', 'Solid', 'Black', 'Dark', 'Black', 'Suit', 'Ladieswear', 'Ladieswear', 'Womens Tailoring', 'Dressed']


## Inference

In [7]:
# 需要推理的目标column name和每个column的unique label
def single_column_infer_fn(model, images, column, unique_labels):
    '''
    input: 
        model: CLIP model
        processor: CLIP processor (global).
        images: path of images
        column: the specific column name
        unique_labels: all possible unique labels in this column
    output: 
        probs: classification probabilities
    '''
    infer_prompt = 'The {c} of photo is {label}' # temporary, need to be changed
    col = ' '.join(column.split('_')) # replaced _ by space
    unique_prompts = [infer_prompt.format(c=col, label=l) for l in unique_labels]
    device = model.device
    images = [Image.open(i) for i in images]
    
    inputs = processor(
        text=unique_prompts,
        images=images,
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    pixel_values = inputs['pixel_values'].to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask if attention_mask is not None else None,
            pixel_values=pixel_values
        )
        logits_per_image = outputs.logits_per_image
    return logits_per_image

In [8]:
def compute_loss_and_eval(model, val_dl, columns_unique_labels):
    '''
    columns_unique_labels: {'column_name1': ['unique_label_name'], 'column_name2': ['unique_label_name']}.
    val_dl: 'image', 'column1', 'column2' ...
    '''
    device = model.device
    print(f'{len(columns_unique_labels)} column need to be evaluated')
    
    all_cols_results = {}
    column_id = 0
    for column, unique_labels in columns_unique_labels.items():
        column_id += 1
        print(f'\nEvaluation on column: {column}')
        all_preds = []
        all_labels = []
        single_column_loss = 0
        for batch in tqdm(val_dl):
            # batch[0] are batched images
            logits_per_image = single_column_infer_fn(model, batch[0], column, unique_labels)
            
            probs = F.softmax(logits_per_image, dim=-1)
            preds = [int(idx) for idx in probs.argmax(dim=-1).cpu().numpy()]
            # batch[1] ~ batch[11] are labels from different columns
            labels = batch[column_id]
            # is necessary to compute loss?
            # 将字符串标签转换为索引
            label_indices = [unique_labels.index(label) for label in labels]
            all_labels.extend(label_indices)
            label_indices = torch.tensor(label_indices, 
                                       device=device)
            # 计算交叉熵损失
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits_per_image, label_indices)
            single_column_loss += loss.item() * len(batch[column_id])
            all_preds.extend(preds)
        
        accuracy = sum(p == l for p, l in zip(all_preds, all_labels)) / len(all_labels)
        single_column_loss = single_column_loss / len(all_labels)
        
        single_col_results = {}
        single_col_results['accuracy'] = accuracy
        single_col_results['loss'] = single_column_loss
        single_col_results['preds'] = all_preds
        single_col_results['labels'] = all_labels
        all_cols_results[column] = single_col_results
        
        print(f'Model accuracy on {column}: {accuracy}')
        print(f'Loss on {column}: {single_column_loss}')
    return all_cols_results
    

In [1]:
# # original model evaluation on validate dataset
# val_dl = DataLoader(val_ds, batch_size=256)
# tmp = compute_loss_and_eval(model, val_dl, columns_unique_labels)

In [9]:
def save_dict_to_json(dictionary, filepath, indent=4, ensure_ascii=False):
    """
    将字典保存为JSON文件
    
    Args:
        dictionary: 要保存的字典
        filepath: 保存路径
        indent: 缩进空格数
        ensure_ascii: 是否确保ASCII编码（False则支持中文）
    """
    # 确保目录存在
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    # 保存为JSON
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(dictionary, f, indent=indent, ensure_ascii=ensure_ascii)

# tmp = {'1':{'loss':1, 'acc':1, 'preds':[1,2,3,4]}, '666':{'loss':1, 'acc':1, 'labels':[1,2,3,4]}, '2':{'loss':1, 'acc':1, 'preds':[1,2,3,4], 'labels':[1,2,3,4]}}
# save_dict_to_json(tmp, '/kaggle/working/epoch0.json')

## Training loop func

In [25]:
def save_clip_checkpoint(model, optimizer, epoch, loss, config=None, save_dir='checkpoints'):
    """
    保存CLIP模型检查点
    
    Args:
        model: CLIP模型
        optimizer: 优化器
        epoch: 当前轮次
        loss: 当前loss
        config: 训练配置
        save_dir: 保存目录
    """
    # 创建保存目录
    os.makedirs(save_dir, exist_ok=True)
    
    # 生成文件名，包含时间戳和epoch信息
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'clip_checkpoint_epoch{epoch}_{timestamp}.pth'
    filepath = os.path.join(save_dir, filename)
    
    # 准备要保存的数据
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'config': config,
        'timestamp': timestamp
    }
    
    # 保存模型
    torch.save(checkpoint, filepath)
    
    # 保存配置信息为单独的JSON文件（便于查看）
    config_file = os.path.join(save_dir, f'config_epoch{epoch}_{timestamp}.json')
    with open(config_file, 'w', encoding='utf-8') as f:
        json.dump(config, f, indent=4, ensure_ascii=False)
    
    print(f"Checkpoint saved: {filepath}")
    return filepath

def train(
    model=None, 
    train_ds=None, 
    val_ds=None, 
    n_epochs=3, 
    batch_size=128, 
    lr=1e-5, 
    infer_fn=None,
    project_name="H&M",  # wandb project name
    run_name='10.09',  # optional wandb run name
):
    # Initialize wandb
    wandb.init(
        project=project_name,
        name=run_name,
        config={
            "learning_rate": lr,
            "epochs": n_epochs,
            "batch_size": batch_size
        }
    )

    train_dl = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn_train, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=batch_size)
    optimizer = AdamW(model.parameters(), lr=lr)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    global_step = 0
    print('Training Begin...')
    # 中间结果需要保存到output中
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0    
        for batch in tqdm(train_dl):
            input_ids = batch['input_ids'].to(device)
            pixel_values = batch['pixel_values'].to(device)
            
            outputs = model(input_ids=input_ids, pixel_values=pixel_values)
            logits_per_image = outputs.logits_per_image
            logits_per_text = outputs.logits_per_text
            
            labels = torch.arange(logits_per_image.size(0)).to(device)
            loss_img = F.cross_entropy(logits_per_image, labels)
            loss_txt = F.cross_entropy(logits_per_text, labels)
            loss = (loss_img + loss_txt) / 2
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            global_step += 1

            # Log training loss every 100 steps
            if global_step % 10 == 0:
                wandb.log({
                    "train/loss": running_loss / 10,
                    "train/step": global_step
                })
                running_loss = 0.0
        
        # save model checkpoints
        save_clip_checkpoint(model, optimizer, epoch, loss)
        
        model.eval()
        
        # evaluations on all columns take about 2.5 hours, if you don't want to evaluate, comment it
        all_cols_res = compute_loss_and_eval(model, val_dl, columns_unique_labels)
        save_dict_to_json(all_cols_res, f'/kaggle/working/epoch-{epoch + 1}.json')
        # Log validation metrics
        log_dict = {
            f"val/epoch": epoch,
        }
        
        # Log individual column metrics
        for column, d in all_cols_res.items():
            log_dict.update({
                f"val/loss_{column}": d['loss'],
                f"val/accuracy_{column}": d['accuracy']
            })
        
        wandb.log(log_dict)
        
        print(f"Epoch {epoch}, Loss: {loss.item()}")
        print(f"Evaluation results on val_dl:\n {acc_loss_dict}")
    
    # Close wandb run
    wandb.finish()

## Ready to Train

In [26]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [27]:
train_arg = {
    'model': peft_model,
    'train_ds': train_ds,
    'val_ds': val_ds,
    'n_epochs': 3,
    'batch_size': 64,
    'lr': 2e-5,
    'infer_fn': None,
    'run_name': 'test',
}
wandb.login(key='2b1626edceae9b68a67d66923587da64398da02c')
train(**train_arg)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Training Begin...


  0%|          | 0/1314 [00:00<?, ?it/s]

Checkpoint saved: checkpoints/clip_checkpoint_epoch0_20241010_152143.pth



KeyboardInterrupt

